In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import Packages.ClusteringHelper as ch
import numpy as np
from tqdm import tqdm
from collections import Counter
import string
import re
from nltk.corpus import stopwords

In [7]:
text, data = ch.read_aida_yago_conll(
    "D:\\Sgmon\\Documents\\Magistrale\\TESI\\ClusteringAndLinking\\aida-yago2-dataset\\AIDA-YAGO2-dataset.tsv")
save = False
if save:
    text_file = open('text.txt', 'w')
    text_file.write(text)
    text_file.close()
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,Bobby,Bobby_Charlton,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [8]:
data = ch.filter_data(data, 3)
data

,documents,tokens,indexes,word_indexes,mentions,entities,wikidatas,numeric_codes,alpha_codes
0,0,EU,"(0, 2)",0,,,,,
1,0,rejects,"(3, 10)",1,,,,,
2,0,German,"(11, 17)",2,German,Germany,http://en.wikipedia.org/wiki/Germany,11867,/m/0345h
3,0,call,"(18, 22)",3,,,,,
4,0,to,"(23, 25)",4,,,,,
...,...,...,...,...,...,...,...,...,...
285584,1392,younger,"(1342, 1349)",265,,,,,
285585,1392,brother,"(1350, 1357)",266,,,,,
285586,1392,",","(1358, 1359)",267,,,,,
285587,1392,Bobby,"(1360, 1365)",268,,,http://en.wikipedia.org/wiki/Bobby_Charlton,4224,/m/01c8x


In [9]:
# Gold standard
ents_data = data[data['entities'] != '']
entities = ents_data['entities'].values
mentions = ents_data['tokens'].values
docs = ents_data['documents'].values
word_indexes = ents_data['word_indexes'].values
unique_entities = np.unique(entities)
gold_standard = Counter(entities)
gold_standard = dict(sorted(gold_standard.items(), key=lambda item: item[1], reverse=True))

#### Gensim word2vec

In [10]:
lowercase = True
remove_stopwords = True
remove_numbers = True
remove_symbols = True

text_preprocessed = text
stop_words = set(stopwords.words("english"))

if remove_symbols:
    text_preprocessed = [x.translate(str.maketrans('', '', string.punctuation)) for x in text_preprocessed]

# rimuovo numeri e punteggiatura
if lowercase:
    text_splitted_training = [x.lower().split() for x in text_preprocessed]
    text_splitted = [x.lower().split() for x in text]
else:
    text_splitted_training = [x.split() for x in text_preprocessed]
    text_splitted = [x.split() for x in text]

if remove_numbers:
    text_splitted_training = [[y for y in x if not y.isnumeric()] for x in text_splitted_training]

if remove_stopwords:
    text_splitted_training = [[y for y in x if y.lower() not in stop_words] for x in text_splitted_training]

Model selection

In [11]:
# create the model
import os
from gensim.models import KeyedVectors
from gensim.models.word2vec import Word2Vec
from multiprocessing import cpu_count

import os
from gensim.models import KeyedVectors

model_path = "D:\Sgmon\Documents\Magistrale\TESI\gensim-data"
"""
sg 0 = CBOW
sh 1 = SKIPGRAM
"""
model = Word2Vec(text_splitted_training, min_count=5, workers=cpu_count(), window=1, sg=0, size=25)

# or select it
#
# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'word2vec-google-news-300', "word2vec-google-news-300.gz"),
#                                                        binary=True)

# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'glove-twitter-25', 'glove-twitter-25.gz'))

# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-50.gz'))


# model = KeyedVectors.load_word2vec_format(
#     os.path.join(model_path, 'glove-wiki-gigaword-300', "glove-wiki-gigaword-300.txt"))

# model = KeyedVectors.load_word2vec_format(os.path.join(model_path, 'fasttext-wiki-news-subwords-300', "fasttext-wiki-news-subwords-300.gz"),
#                                                        binary=False)

In [12]:
leven_cluster = np.loadtxt("../aida-yago2-dataset/db_cluster_levestein_0_3.txt", dtype=np.int32)
lev_cluster_dict = {}
windows = 25
doc_windows = 20
for i, x in enumerate(leven_cluster):
    try:
        lev_cluster_dict[x].append(
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows)))
    except:
        lev_cluster_dict[x] = [
            (mentions[i], entities[i], docs[i], word_indexes[i],
             ch.get_context(text_splitted[docs[i]], word_indexes[i], windows=windows))]
# """se uso un cluster diverso da minimo 0 allora scommento"""
# del lev_cluster_dict[-1]

In [13]:
cluster_all = []
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

# from hdbscan import HDBSCAN

clusterizator = AgglomerativeClustering(n_clusters=None, affinity='cosine', distance_threshold=0.013, linkage="single")
# clusterizator = DBSCAN(min_samples=0, metric="cosine", n_jobs=-1, eps=0.1)
# clusterizator = HDBSCAN(min_cluster_size=2, metric="euclidean", min_samples=1, cluster_selection_epsilon=0.4)

for x in tqdm(lev_cluster_dict):
    contexts_vectorized = ch.calculate_context_vector(model, lev_cluster_dict[x], text_splitted, doc_window=doc_windows,
                                                      doc_context_alpha=0.3, word_context_aplha=.7)
    if len(contexts_vectorized) > 1:
        cluster = clusterizator.fit_predict(contexts_vectorized)
    else:
        cluster = np.zeros(1)
    cluster_all.append(cluster)
    # print(cluster_all)
    # break

100%|██████████| 2750/2750 [00:08<00:00, 327.61it/s]


In [14]:
clustered_entities = [[row[1] for row in lev_cluster_dict[index]] for index in lev_cluster_dict.keys()]

In [15]:
clusters_splitted = []
for n_cluster in range(len(clustered_entities)):
    cluster_unique = dict.fromkeys(set(cluster_all[n_cluster]), [])
    for key in cluster_unique:
        cluster_unique[key] = [clustered_entities[n_cluster][index] for index in
                               range(len(clustered_entities[n_cluster])) if cluster_all[n_cluster][index] == key]
    clusters_splitted.append(cluster_unique)

In [16]:
clusters_splitted_dict = [dict(Counter(x[y])) for x in clusters_splitted for y in x]

### Gold standard

In [17]:
golden_standard_dict = ch.get_gold_standard_dict(data)

### B-CUBED

In [18]:
# B-cubed - precision
bcubed_precision_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for cluster in clusters_splitted_dict:
        try:
            bcubed_precision_num = bcubed_precision_num + (pow(cluster[gold_key], 2) /
                                                           sum([x for x in cluster.values()]))
        except:
            pass
bcubed_precision = bcubed_precision_num / sum([y for x in clusters_splitted_dict for y in x.values()])
bcubed_precision

100%|██████████| 2089/2089 [00:01<00:00, 1609.31it/s]


0.8818219807708759

In [19]:
# B-cubed - recall
bcubed_recall_num = 0
for gold_key in tqdm(golden_standard_dict.keys()):
    for cluster in clusters_splitted_dict:
        try:
            bcubed_recall_num = bcubed_recall_num + (pow(cluster[gold_key], 2) /
                                                     golden_standard_dict[gold_key])
        except:
            pass
bcubed_recall = bcubed_recall_num / ents_data.shape[0]
bcubed_recall

100%|██████████| 2089/2089 [00:01<00:00, 1444.61it/s]


0.7888689224921338

In [20]:
bcubed_f1 = (2 * (bcubed_recall * bcubed_precision)) / (
        bcubed_precision + bcubed_recall)
bcubed_f1

0.8327596139321148

### evaluation levestein

CEAFm precision

In [22]:
max_evaluation_dict = ch.get_optimal_alignment(clusters_splitted_dict, set(entities), is_dict=False)


In [23]:
# CEAFm_precision
CEAFm_precision = sum([x for x in max_evaluation_dict.values()]) / sum(
    golden_standard_dict.values())
CEAFm_precision

0.7864357244165556

CEAFm recall

In [24]:
# CEAFm_recall
CEAFm_recall = sum([x for x in max_evaluation_dict.values()]) / ents_data.shape[0]
CEAFm_recall

0.7864357244165556

CEAFm Fscore

In [25]:
# CEAFm_f1
CEAFm_f1 = (2 * (CEAFm_recall * CEAFm_precision)) / (
        CEAFm_precision + CEAFm_recall)
CEAFm_f1

0.7864357244165556

In [26]:
model.most_similar('italy')

D:\Programmi\Anaconda\envs\entity_clustering\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('market', 0.9996877908706665),
 ('gave', 0.9995535612106323),
 ('ended', 0.9995139837265015),
 ('belgium', 0.9995114803314209),
 ('mark', 0.9994690418243408),
 ('added', 0.9994654655456543),
 ('group', 0.9994617700576782),
 ('france', 0.9994571208953857),
 ('britain', 0.9994548559188843),
 ('sweden', 0.9994432926177979)]